### Edgar Moises Hernandez-Gonzalez
#### 29-04-20-26-08-20
#### Clasificacion de imagenes motoras en EEG con CNN-1D + LSTM o CNN-1D + GRU
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Espectrogramas STFFT (3D)

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import numpy as np
import pandas as pd
import time
from scipy.signal import spectrogram
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import cohen_kappa_score, confusion_matrix
# from keras.utils import to_categorical
from keras.models import load_model

Using TensorFlow backend.


In [2]:
x_train = pd.read_csv("../Datos/MI-EEG-A09T.csv", header=None)
x_test = pd.read_csv("../Datos/MI-EEG-A09E.csv", header=None)
y_test = pd.read_csv("../Datos/EtiquetasA09E.csv", header=None)

In [3]:
print(x_train.shape)
print(x_test.shape)
print(y_test.shape)

(288, 22000)
(288, 22000)
(288, 1)


In [4]:
# esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test
n_classes = len(np.unique(y_test))

print("n_classes:", n_classes)

n_classes: 4


In [ ]:
#codificar etiquetas con valores de 0 a nClases-1
# y_test = LabelEncoder().fit_transform(y_test)

In [5]:
#unir 22 canales y resize de tamaño proporcional
def unir_espectrogramas_vertical(data):
  fs = 250
  datos = np.zeros((data.shape[0],990,31))
  temporal = np.zeros((990,31))
  for i in range(data.shape[0]): #288 muestras
    for j in range(22): #22 canales
      sig = data.iloc[i, j*1000:(j+1)*1000]
      #sig = sig[124:624]
      #espectrograma de STFFT
      f, t, Sxx = spectrogram(sig, fs=250, window='hann', nperseg=250,
                              noverlap=225, nfft=500, scaling='spectrum')
      temporal[j*45:(j+1)*45, :] = Sxx[16:61, :]
    datos[i] = temporal
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [6]:
#llamar a unir_espectrogramas_vertical(data)
inicio = time.time()

x_train = unir_espectrogramas_vertical(x_train)
x_test = unir_espectrogramas_vertical(x_test)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
0
100
200
Tiempo: 8.265802145004272


In [7]:
print(x_test.shape)

(288, 990, 31)


In [8]:
x = []
x.append(np.ceil(np.max(x_train)))
# esta linea solo se ocupa si se va a predecir un modelo creado con CV
# x.append(np.ceil(np.max(x_test)))

In [9]:
#convertir a float
x_test = x_test.astype('float32')

#escalar los valores en un rango de 0 a 1 (normalizar)
x_test /= np.ceil(np.max(x))

In [10]:
#convertir de 3D a 5D
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1], x_test.shape[2], 1))

print(x_test.shape)

(288, 1, 990, 31, 1)


In [11]:
model = load_model('../Modelos/RNN_STFT09.hdf5')

In [12]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_21 (TimeDis (None, 1, 990, 31, 16)    160       
_________________________________________________________________
time_distributed_22 (TimeDis (None, 1, 495, 15, 16)    0         
_________________________________________________________________
time_distributed_23 (TimeDis (None, 1, 495, 15, 16)    2320      
_________________________________________________________________
time_distributed_24 (TimeDis (None, 1, 247, 7, 16)     0         
_________________________________________________________________
time_distributed_25 (TimeDis (None, 1, 27664)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 8)                 885536    
_________________________________________________________________
dense_9 (Dense)              (None, 128)              

In [15]:
inicio = time.time()

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.2076101303100586


In [ ]:
print("loss: %.4f" % test_loss)
print("accuracy: %.4f" % test_acc)

loss: 0.6675
accuracy: 0.9375


In [ ]:
probabilidades = model.predict(x_test)
y_pred = np.argmax(probabilidades, 1) #1=fila

In [ ]:
kappa = cohen_kappa_score(y_test, y_pred)
print("kappa: %.4f " % kappa)

kappa: 0.8750 


In [ ]:
matriz_confusion = confusion_matrix(y_test, y_pred)
print("confusion matrix:\n", matriz_confusion)

confusion matrix:
 [[66  6]
 [ 3 69]]


In [16]:
inicio = time.time()

probabilidades = model.predict(x_test[0].reshape(1,1,990,31,1))
y_pred = np.argmax(probabilidades, 1) #1=fila

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.004952907562255859
